In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("Bibliotecas importadas!")

In [ ]:
try:
    # Montando os dataframes
    path_transacoes = '../data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
    path_produtos = '../part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet'
    path_pdvs = '../data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet'

    df_transacoes = pd.read_parquet(path_transacoes)
    df_produtos = pd.read_parquet(path_produtos)
    df_pdvs = pd.read_parquet(path_pdvs)

    print("Arquivos Parquet carregados com sucesso!")

    # Unindo os dataframes
    df_completo = pd.merge(df_transacoes, df_produtos, on='produto', how='left')
    df_completo = pd.merge(df_completo, df_pdvs, on='pdv', how='left')

    print("DataFrames unidos!")
    
    # Mostra as 5 primeiras linhas para confirmar que tudo funcionou
    display(df_completo.head())

except FileNotFoundError:
    print("ERRO: Arquivos Parquet não encontrados na pasta /data/. Verifique se você moveu os arquivos para lá.")